In [1]:
# # Solve the following MIP:
# #  maximize
# #        x +   y + 2 z
# #  subject to
# #        x + 2 y + 3 z <= 4
# #        x +   y       >= 1
# #        x, y, z binary

# import gurobipy as gp

# # Create a new model
# m = gp.Model()

# # Create variables
# x = m.addVar(vtype='B', name="x")
# y = m.addVar(vtype='B', name="y")
# z = m.addVar(vtype='B', name="z")

# # Set objective function
# m.setObjective(x + y + 2 * z, gp.GRB.MAXIMIZE)

# # Add constraints
# m.addConstr(x + 2 * y + 3 * z <= 4)
# m.addConstr(x + y >= 1)

# # Solve it!
# m.optimize()

# print(f"Optimal objective value: {m.objVal}")
# print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")

In [2]:
import gurobipy as gp

# Create a new model
m = gp.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [3]:
# Initialize problem parameters
scraps = 3
boxes = 6

set_scraps = range(0, scraps)
set_boxes = range(0, boxes + 1)

# Box distance matrix
d = [[0, 1, 2, 3, 4, 5, 6],
    [1, 0, 1, 2, 3, 4, 5],
    [2, 1, 0, 1, 2, 3, 4],
    [3, 2, 1, 0, 1, 2, 3],
    [4, 3, 2, 1, 0, 1, 2],
    [5, 4, 3, 2, 1, 0, 1],
    [6, 5, 4, 3, 2, 1, 0]]

# Distance from hub
d_hub = [0, 1, 2, 3, 4, 5, 6]

recipes = [[0, 1], [1, 2], [2, 0]]

In [4]:
# Create variables

x = {}
for scrap in set_scraps:
    for box in set_boxes:
        x[scrap, box] = m.addVar(vtype='B', name=f"x[{scrap},{box}]")

x

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 4): <gurobi.Var *Awaiting Model Update*>,
 (0, 5): <gurobi.Var *Awaiting Model Update*>,
 (0, 6): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (1, 6): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 6): <gurobi.Var *Awaiting Model Update*>}

In [5]:
# Add constraints

# Each scrap is assigned to at least one box
for s in set_scraps:
    m.addConstr(sum(x[s, b] for b in set_boxes) >= 1)

# Each box is assigned at most one scrap
for b in set_boxes[1:]:
    m.addConstr(sum(x[s, b] for s in set_scraps) == 1)

# Dummy box is empty
m.addConstr(sum(x[s, 0] for s in set_scraps) == 0)


<gurobi.Constr *Awaiting Model Update*>

In [6]:
# Set objective function

# Test
#objective = gp.quicksum(x[1, b] for b in set_boxes)

# Simplified objective
objective = gp.quicksum(gp.quicksum(d_hub[b] * x[s, b] for b in set_boxes for s in recipe) for recipe in recipes)

m.setObjective(objective, gp.GRB.MINIMIZE)

In [7]:
# Solve it!
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 PRO 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 10 rows, 21 columns and 42 nonzeros
Model fingerprint: 0x4323a326
Variable types: 0 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 42.0000000
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolved: 9 rows, 18 columns, 36 nonzeros
Variable types: 0 continuous, 18 integer (18 binary)

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 1: 42 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.200000000000e+01, best bound

In [8]:
# Show solution
print(f"Optimal objective value: {m.objVal}")

for v in m.getVars():
    if v.X > 0:
        print(f"{v.VarName} = {v.X}")

Optimal objective value: 42.0
x[0,6] = 1.0
x[1,2] = 1.0
x[1,4] = 1.0
x[1,5] = 1.0
x[2,1] = 1.0
x[2,3] = 1.0
